# Comparison checks for quality control flags
Covers QC8-12

## Table of contents
[QC8 R99p](#QC8---R99p)  
[QC9 PRCPTOT](#QC9---PRCPTOT)  
[QC10 World Record](#QC10---World-Record)  
[QC11 Rx1day](#QC11---Rx1day)  
[QC12 CDD](#QC12---CDD)  

See '3.3.1 Suspiciously high values' in Lewis et al. (2021)

# QC8 - R99p 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC9 - PRCPTOT 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC10 - World Record 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC11 - Rx1day 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 

# QC12 - CDD 
[Back to Index](#Table-of-contents)

#### Differences from `intense-qc`: 
- 